<a href="https://colab.research.google.com/github/Anticsss/models/blob/master/implementing_PubTabNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [1]:
# importing prerequisites
import sys
import requests
import tarfile
import numpy as np
import os
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
import requests
from matplotlib import pyplot as plt
%matplotlib inline

try:
    import jsonLines
except:
    !pip install jsonlines
    import jsonlines

import tensorflow as tf

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tqdm import tqdm
import re
import time
import json
from glob import glob
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, GlobalAveragePooling2D, Add, Dense, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

## Download and Extract the Dataset

Since the dataset is large (~12GB), here we will be downloading a small subset of the data and extract it. 

In [2]:
#!wget -O PubTabNet.tar.gz https://dax-cdn.cdn.appdomain.cloud/dax-pubtabnet/2.0.0/pubtabnet.tar.gz

In [3]:
fname = 'examples.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-pubtabnet/2.0.0/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

386348

In [4]:
# Extracting the dataset
#fname="/content/PubTabNet.tar.gz"
tar = tarfile.open(fname)
tar.extractall()
tar.close()

In [5]:
# Verifying the file was extracted properly
data_path = "examples/"
path.exists(data_path)

True

## Visualizing the Data

In this section, we visualize the raw image and extract it's HTML annotation from the JSON file. 
We further render the table using Jupyter notebook's inbuilt HTML capabilities. 

In [6]:
# Helper function to read in tables from the annotations
from bs4 import BeautifulSoup as bs
from html import escape

def format_html(img):
    ''' Formats HTML code from tokenized annotation of img
    '''
    html_code = img['html']['structure']['tokens'].copy()
    to_insert = [i for i, tag in enumerate(html_code) if tag in ('<td>', '>')]
    for i, cell in zip(to_insert[::-1], img['html']['cells'][::-1]):
        if cell['tokens']:
            cell = [escape(token) if len(token) == 1 else token for token in cell['tokens']]
            cell = ''.join(cell)
            html_code.insert(i + 1, cell)
    html_code = ''.join(html_code)
    html_code = '''<html>
                   <head>
                   <meta charset="UTF-8">
                   <style>
                   table, th, td {
                     border: 1px solid black;
                     font-size: 10px;
                   }
                   </style>
                   </head>
                   <body>
                   <table frame="hsides" rules="groups" width="100%%">
                     %s
                   </table>
                   </body>
                   </html>''' % html_code

    # prettify the html
    soup = bs(html_code)
    html_code = soup.prettify()
    return html_code

In [7]:
# Loading an example annotation
with jsonlines.open('examples/PubTabNet_Examples.jsonl', 'r') as reader:
    imgs = list(reader)

In [8]:
imgs[0]['filename']

'PMC4840965_004_00.png'

In [9]:
# Showing the raw image
from IPython.display import Image as displayImage
filename = imgs[0]['filename']
displayImage("examples/"+filename)

In [10]:
# Extracting the HTML for the table from the annotation
html_string = format_html(imgs[0])
print(html_string)

<html>
 <head>
  <meta charset="utf-8"/>
  <style>
   table, th, td {
                     border: 1px solid black;
                     font-size: 10px;
                   }
  </style>
 </head>
 <body>
  <table frame="hsides" rules="groups" width="100%">
   <thead>
    <tr>
     <td>
      <b>
       Variable
      </b>
     </td>
     <td>
      <b>
       Hazard ratio
      </b>
     </td>
     <td>
      <b>
       95 % CI
      </b>
     </td>
     <td>
      <b>
       <i>
        p
       </i>
       value*
      </b>
     </td>
    </tr>
   </thead>
   <tbody>
    <tr>
     <td>
      Age (median)
     </td>
     <td>
     </td>
     <td>
     </td>
     <td>
      0.716
     </td>
    </tr>
    <tr>
     <td>
      ≤69
     </td>
     <td>
      1.000
     </td>
     <td>
     </td>
     <td>
     </td>
    </tr>
    <tr>
     <td>
      &gt;69
     </td>
     <td>
      0.839
     </td>
     <td>
      0.310–2.268
     </td>
     <td>
     </td>
    </tr>
    <tr>
     <td>
 

In [11]:
# Rendering the above HTML in Jupyter Notebook for a more readable format
from IPython.core.display import display, HTML
display(HTML(html_string))

Variable,Hazard ratio,95 % CI,p value*
Age (median),,,0.716
≤69,1.000,,
>69,0.839,0.310–2.268,
Gender,,,0.142
Male,1.000,,
Female,0.426,0.152–1.190,
Type of surgery,,,0.010
Low anterior resection,1.000,,
Abdominoperineal resection,3.140,0.919–10.725,
Tumor location,,,0.710


In [12]:
imgs[0]

{'filename': 'PMC4840965_004_00.png',
 'html': {'cells': [{'bbox': [1, 4, 27, 13],
    'tokens': ['<b>', 'V', 'a', 'r', 'i', 'a', 'b', 'l', 'e', '</b>']},
   {'bbox': [219, 4, 260, 13],
    'tokens': ['<b>',
     'H',
     'a',
     'z',
     'a',
     'r',
     'd',
     ' ',
     'r',
     'a',
     't',
     'i',
     'o',
     '</b>']},
   {'bbox': [336, 4, 363, 13],
    'tokens': ['<b>', '9', '5', ' ', '%', ' ', 'C', 'I', '</b>']},
   {'bbox': [456, 4, 484, 13],
    'tokens': ['<b>',
     '<i>',
     'p',
     '</i>',
     ' ',
     'v',
     'a',
     'l',
     'u',
     'e',
     '*',
     '</b>']},
   {'bbox': [1, 17, 46, 27],
    'tokens': ['A', 'g', 'e', ' ', '(', 'm', 'e', 'd', 'i', 'a', 'n', ')']},
   {'tokens': []},
   {'tokens': []},
   {'bbox': [456, 17, 475, 27], 'tokens': ['0', '.', '7', '1', '6']},
   {'bbox': [8, 31, 23, 41], 'tokens': [' ', '≤', '6', '9']},
   {'bbox': [219, 31, 238, 41], 'tokens': ['1', '.', '0', '0', '0']},
   {'tokens': []},
   {'tokens': []},
  

In [13]:
PATH = os.path.abspath('.') +"/"+ data_path
PATH

'/content/examples/'

In [14]:
# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for img in imgs:
    caption = '<start> ' + ' '.join(img['html']['structure']['tokens']) + ' <end>'
    image_id = img['filename']
    full_image_path = PATH + 'examples/' + image_id

    all_img_name_vector.append(full_image_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# Select the first 30000 captions from the shuffled set
#num_examples = 30000
#train_captions = train_captions[:num_examples]
#img_name_vector = img_name_vector[:num_examples]
train_captions, img_name_vector

(['<start> <thead> <tr> <td> </td> <td  colspan="5" > </td> <td  colspan="5" > </td> <td> </td> </tr> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> </thead> <tbody> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> <tr> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> <td> </td> </tr> <tr> <td> </td> <td> </

In [15]:
len(train_captions), len(img_name_vector)

(20, 20)

### Data Preprocessing

#### Preprocess the images using InceptionV3
Next, we will use InceptionV3 (which is pretrained on Imagenet) to classify each image. We will preprocess images.

First, we will reshape the images into expected format by:

* Resizing the image to 448px by 448px


*   Preprocess the images using the preprocess_input method to normalize the image so that it contains pixels in the range of -1 to 1.

In [16]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [17]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

## Caching the features extracted from InceptionV3

We will normalize each channel by z-score of each image and cache the output to disk. Caching the output in RAM would be faster but also memory intensive, requiring 448 \* 448 \* 3 floats per image. At the time of writing, this exceeds the memory limitations of Colab (currently 12GB of memory).

Performance could be improved with a more sophisticated caching strategy (for example, by sharding the images to reduce random access disk I/O), but that would require more code.

The caching will take about 10 minutes to run in Colab with a GPU. If you'd like to see a progress bar, you can: 

1. install [tqdm](https://github.com/tqdm/tqdm):

    `!pip install tqdm`

2. Import tqdm:

    `from tqdm import tqdm`

3. Change the following line:

    `for img, path in image_dataset:`

    to:

    `for img, path in tqdm(image_dataset):`


In [27]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(1)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  print(batch_features.shape)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))
  print(batch_features.shape)
  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)
(1, 5, 5, 2048)
(1, 25, 2048)


In [28]:
"""
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(10)

for img, path in tqdm(image_dataset):
  #print(img.shape)
  batch_features = tf.image.per_image_standardization(img)
  print(batch_features.shape)

  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))
  print(batch_features.shape)  
  
  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
"""

'\n# Get unique images\nencode_train = sorted(set(img_name_vector))\n\n# Feel free to change batch_size according to your system configuration\nimage_dataset = tf.data.Dataset.from_tensor_slices(encode_train)\nimage_dataset = image_dataset.map(\n  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(10)\n\nfor img, path in tqdm(image_dataset):\n  #print(img.shape)\n  batch_features = tf.image.per_image_standardization(img)\n  print(batch_features.shape)\n\n  batch_features = tf.reshape(batch_features,\n                              (batch_features.shape[0], -1, batch_features.shape[3]))\n  print(batch_features.shape)  \n  \n  for bf, p in zip(batch_features, path):\n    path_of_feature = p.numpy().decode("utf-8")\n    np.save(path_of_feature, bf.numpy())\n'

## Preprocess and tokenize the captions

* First, you'll tokenize the captions (for example, by splitting on spaces). This gives us a  vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).
* Next, you'll limit the vocabulary size to the top 5,000 words (to save memory). You'll replace all other words with the token "UNK" (unknown).
* You then create word-to-index and index-to-word mappings.
* Finally, you pad all sequences to be the same length as the longest one.

In [29]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [30]:
# Choose the top 5000 words from the vocabulary
top_k = 32
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [31]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [32]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [33]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [34]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

In [35]:
max_length

869

## Create a tf.data dataset for training


 Our images and captions are ready! Next, let's create a tf.data dataset to use for training our model.

In [36]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 1
BUFFER_SIZE = 1000
embedding_dim = 16
units = 12
vocab_size = top_k + 1
num_steps = len(img_name_vector) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 1

In [37]:
# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [38]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_vector, cap_vector))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Model

Fun fact: the decoder below is identical to the one in the example for [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb).

The model architecture is inspired by the [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf) paper.

* This image is then passed through the CNN Encoder (which consists of a single Fully connected layer).
* The RNN (here GRU) attends over the image to predict the next word.

In [39]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [40]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [41]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [42]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [43]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [44]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [45]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

## Training

* You extract the features stored in the respective `.npy` files and then pass those features through the encoder.
* The encoder output, hidden state(initialized to 0) and the decoder input (which is the start token) is passed to the decoder.
* The decoder returns the predictions and the decoder hidden state.
* The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
* Use teacher forcing to decide the next input to the decoder.
* Teacher forcing is the technique where the target word is passed as the next input to the decoder.
* The final step is to calculate the gradients and apply it to the optimizer and backpropagate.


In [46]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [47]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [ ]:
EPOCHS = 2

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 2 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## Caption!

* The evaluate function is similar to the training loop, except you don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the end token.
* And store the attention weights for every time step.

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


## Try it on your own images
For fun, below we've provided a method you can use to caption your own images with the model we've just trained. Keep in mind, it was trained on a relatively small amount of data, and your images may be different from the training data (so be prepared for weird results!)


In [ ]:
image_path = # input an image path

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)